# GovernedToolExecutor

`GovernedToolExecutor` is the **primary integration point** for adding governance to agent tool execution. It wraps tool calls with full governance evaluation through Nomotic's 14-dimension pipeline, ensuring that every action an agent takes is evaluated, tracked, and auditable.

When an agent invokes a tool, the executor:

1. **Evaluates** the action against the agent's configured scope, trust profile, and all governance dimensions.
2. **Executes** the tool function only if governance approves (verdict: `ALLOW`).
3. **Denies** execution with a structured explanation when the action falls outside the agent's authority.
4. **Updates trust** continuously — successful actions build trust; violations erode it.
5. **Records** every decision in an immutable audit trail for transparency and accountability.

This notebook walks through creating a governed executor, registering tools, executing governed actions, observing trust evolution, and inspecting the audit trail.

In [ ]:
# Install nomotic in development mode and birth an agent identity
!pip install -e "."
!nomotic birth --name executor-agent --role processor

In [ ]:
from nomotic import GovernedToolExecutor, GovernanceRuntime, Action, AgentContext, TrustProfile

# Create a governance runtime
runtime = GovernanceRuntime()

# Configure the scope for executor-agent: allow read, write, and query actions
runtime.configure_scope("executor-agent", {"read", "write", "query"})

print("Runtime created and scope configured.")
print("Allowed actions for executor-agent: read, write, query")

In [ ]:
from nomotic.integrations.tool_adapter import ToolRegistry

# Define three simple mock tool functions
def read_db(query: str) -> str:
    """Read from the database."""
    return f"Results for: {query}"

def write_db(data: str) -> str:
    """Write to the database."""
    return f"Written: {data}"

def delete_db(table: str) -> str:
    """Delete from the database (dangerous operation)."""
    return f"Deleted: {table}"

# Create a GovernedToolExecutor with the runtime
# We use the runtime directly for in-process governance
executor = GovernedToolExecutor("executor-agent", test_mode=True)

# Register each tool in a ToolRegistry
tools = ToolRegistry()
tools.add("read_db", read_db, description="Read from the database", action_type="read")
tools.add("write_db", write_db, description="Write to the database", action_type="write")
tools.add("delete_db", delete_db, description="Delete from the database", action_type="delete")

print(f"Registered tools: {tools.names()}")
print(f"Executor connected for agent: {executor.agent_id}")
print(f"Test mode: {executor.is_test_mode}")

In [ ]:
# Execute governed tools through the executor
# read_db should be ALLOWED ("read" is in scope)
result_read = executor.execute(
    action="read",
    target="customers",
    params={"query": "SELECT * FROM customers"},
    tool_fn=lambda: read_db("SELECT * FROM customers"),
)
print(f"read_db  -> verdict: {result_read.verdict}, allowed: {result_read.allowed}, data: {result_read.data}")

# write_db should be ALLOWED ("write" is in scope)
result_write = executor.execute(
    action="write",
    target="orders",
    params={"data": "new order record"},
    tool_fn=lambda: write_db("new order record"),
)
print(f"write_db -> verdict: {result_write.verdict}, allowed: {result_write.allowed}, data: {result_write.data}")

# delete_db should be DENIED ("delete" is NOT in scope)
result_delete = executor.execute(
    action="delete",
    target="users",
    params={"table": "users"},
    tool_fn=lambda: delete_db("users"),
)
print(f"delete_db -> verdict: {result_delete.verdict}, allowed: {result_delete.allowed}, reason: {result_delete.reason}")
print(f"             data: {result_delete.data}  (None because execution was denied)")

In [ ]:
# Run 20 actions (mix of read and write) and track trust evolution
trust_scores = []

for i in range(20):
    # Alternate between read and write actions
    if i % 2 == 0:
        action_type = "read"
        tool_fn = lambda: read_db(f"query_{i}")
    else:
        action_type = "write"
        tool_fn = lambda: write_db(f"data_{i}")

    result = executor.execute(
        action=action_type,
        target=f"resource_{i}",
        tool_fn=tool_fn,
    )
    trust_scores.append(result.trust_after)

# Print trust score progression
print("Trust score progression over 20 governed actions:")
print("-" * 50)
for i, score in enumerate(trust_scores):
    bar = "#" * int(score * 40)
    print(f"  Action {i+1:2d}: {score:.4f}  {bar}")

In [ ]:
# Inspect the audit trail
# Access the audit summary from the executor
summary = executor.get_audit_summary()
print("Audit Summary")
print("=" * 60)
print(f"Total records: {summary.get('total_records', 'N/A')}")
print(f"By verdict: {summary.get('by_verdict', {})}")
print()

# Display individual audit records with action_type, verdict, and trust_score
# The executor writes to a persistent log store; retrieve the records
print("Audit Trail Records")
print("=" * 60)
print(f"{'#':>3}  {'Action Type':<12}  {'Verdict':<10}  {'Trust Score':>11}")
print("-" * 60)

log_records = executor._log_store.get_records(executor.agent_id)
for idx, record in enumerate(log_records):
    action_type = getattr(record, 'action_type', 'unknown')
    verdict = getattr(record, 'verdict', 'unknown')
    trust_score = getattr(record, 'trust_score', 0.0)
    print(f"{idx+1:3d}  {action_type:<12}  {verdict:<10}  {trust_score:>11.4f}")

## Framework Integration

Nomotic provides ready-made adapters for popular agentic AI frameworks in the `nomotic.integrations` module:

- **CrewAI** (`nomotic.integrations.crewai_adapter`) — Wrap CrewAI tool calls with governance evaluation so that agents operating within a crew respect organizational scope and trust boundaries.
- **LangGraph** (`nomotic.integrations.langgraph_adapter`) — Integrate governance checkpoints into LangGraph node execution, enabling governed state transitions and tool invocations.
- **AutoGen** (`nomotic.integrations.autogen_adapter`) — Add governance to AutoGen's multi-agent conversations, ensuring that each agent's tool use is evaluated against its trust profile and scope.

Each adapter builds on the same `GovernedToolExecutor` core demonstrated in this notebook. For detailed usage, configuration options, and framework-specific examples, refer to the Nomotic documentation.